# A simple bottleneck

Very simple code that :
- make_random_edges : generate a random graph given a number of edge and nodes
- find_unique_edges : filter the list of edges to only keep an instance of each (i.e. remove duplicate edges)
- contains : verify if an edge is part of a list

In [1]:
import random

def make_random_edges(n_edges=100, n_nodes=10):
    random.seed(42)
    edges = [[random.randint(0, n_nodes), random.randint(0, n_nodes)] for e in range(n_edges)]
    return edges

def find_unique_edges(edges):
    edges = list(edges)
    unique_edges = []
    while edges:
        edge = edges.pop()
        if not contains(edges, edge):
            unique_edges.append(edge)
    return unique_edges

def contains(edges, edge):
    for e in edges:
        if sorted(e) == sorted(edge):
            return True
    return False


Some profiling functions to ease the process

In [2]:
from functools import wraps
from cProfile import Profile
from tempfile import NamedTemporaryFile
import pstats

_time_profiles = {}

def profile_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        profile = Profile()
        ret = profile.runcall(func, *args, **kwargs)
        _time_profiles[(wrapper, ) + args] = profile
        
        return ret 
    return wrapper

def profile_stats(profile):
    temp_stats = NamedTemporaryFile(delete_on_close=False)
    profile.dump_stats(temp_stats.name)
    return pstats.Stats(temp_stats.name)

Simple profiling with text outputs  

In [3]:
@profile_time
def remove_duplicate_edges(n_edges=2000):
    edges = make_random_edges(n_edges, 200)
    unique_edges = find_unique_edges(edges)
    return unique_edges

unique_edges = remove_duplicate_edges(2000)

profile = _time_profiles[(remove_duplicate_edges, 2000)]

stats = profile_stats(profile)
stats.strip_dirs()
stats.sort_stats('time')
stats.print_stats()

Wed Oct  2 14:04:18 2024    C:\Users\UTILIS~1\AppData\Local\Temp\tmpklm22zdd

         3942727 function calls in 2.693 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  3903622    1.643    0.000    1.643    0.000 {built-in method builtins.sorted}
     2000    1.022    0.001    2.662    0.001 2053488929.py:17(contains)
     4000    0.006    0.000    0.016    0.000 random.py:291(randrange)
     4000    0.005    0.000    0.008    0.000 random.py:242(_randbelow_with_getrandbits)
        1    0.003    0.003    2.646    2.646 2053488929.py:8(find_unique_edges)
        1    0.003    0.003    0.022    0.022 2053488929.py:3(make_random_edges)
     4000    0.003    0.000    0.019    0.000 random.py:332(randint)
    12000    0.002    0.000    0.002    0.000 {built-in method _operator.index}
     4000    0.001    0.000    0.001    0.000 {method 'bit_length' of 'int' objects}
     5051    0.001    0.000    0.001    0.000 {method 'getran

Advance profiling with visualization

In [4]:
%reload_ext snakeviz

In [5]:
import subprocess

def display_stats(profile):
    with NamedTemporaryFile(delete=False, suffix='.prof') as temp_stats:
        profile.dump_stats(temp_stats.name)
        temp_stats.close()
        subprocess.run(['snakeviz', temp_stats.name])

In [6]:
"""@profile_time
def remove_duplicate_edges(n_edges=2000):
    edges = make_random_edges(n_edges, 200)
    unique_edges = find_unique_edges(edges)
    return unique_edges

unique_edges = remove_duplicate_edges(2000)

profile = _time_profiles[(remove_duplicate_edges, 2000)]
display_stats(profile)"""

'@profile_time\ndef remove_duplicate_edges(n_edges=2000):\n    edges = make_random_edges(n_edges, 200)\n    unique_edges = find_unique_edges(edges)\n    return unique_edges\n\nunique_edges = remove_duplicate_edges(2000)\n\nprofile = _time_profiles[(remove_duplicate_edges, 2000)]\ndisplay_stats(profile)'

Propose a new implementation and profile it

In [7]:
def find_unique_edges_better(edges):
    print(len(edges))
    unique_edges = {}
    unique_edges.update(list(edges))
    unique_edges = list(unique_edges)
    print(len(unique_edges))
    return unique_edges

@profile_time
def remove_duplicate_edges_better(n_edges=2000):
    edges = make_random_edges(n_edges, 200)
    unique_edges = find_unique_edges_better(edges)
    return unique_edges

unique_edges = remove_duplicate_edges_better(2000)

profile = _time_profiles[(remove_duplicate_edges_better, 2000)]
display_stats(profile)

2000
201
